## From EXR to YUV
* .exr images from ftp://svtopencontent.svt.se
* 1500 images at 50 fps = 30sec
* inside a .yuv raw video with 4k resolution and yuv420p sampling

In [2]:
import subprocess
c_seq = "ffmpeg -y -start_number 180000 -framerate 50 -i test_vids/src_midnight/midnight_sun_%04d.exr \
        -c:v rawvideo -pix_fmt yuv420p -s 3840x2160 -r 50 test_vids/midnight.yuv"
subprocess.call(c_seq, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

0

## From YUV to YUV multires
the .yuv reference uncompressed file is downsized to 5 more resolutions

In [ ]:
import numpy as np
import subprocess
source = "smithy"
res_widths = [3840, 2560, 1920, 1280, 1024, 640]
res_heights = [2160, 1440, 1080, 720, 576, 360]
for i in range(1, res_widths.size):
    path = "test_vids/downsampled/" + source + "_" + str(res_widths[i-1]) + ".yuv"
    out = "test_vids/downsampled/" + source + "_" + str(res_widths[i]) + ".yuv"
    command = f"ffmpeg -f rawvideo -video_size {res_widths[i-1]}x{res_heights[i-1]} -r 50 -pixel_format yuv420p \
    -i {path} -f rawvideo -c:v rawvideo -filter:v scale={res_widths[i]}x{res_heights[i]} -pix_fmt yuv420p {out}"
    subprocess.call(command, shell=True)

## From YUV to H264 AVC

In [3]:
import os
import subprocess
source = "forest"
path = "test_vids/refs/" + source + ".yuv"
res_width = 3840
res_height = 2160
quality = [10, 17, 23, 29, 35]
for current_crf in range(0,5):
    out = "test_vids/encoded/" + source + "/" + source + "AVC_cf" + str(quality[current_crf]) + ".mp4"
    to_avc = f"ffmpeg -f rawvideo -video_size {res_width}x{res_height} \
    -r 50 -pixel_format yuv420p -i {path} -c:v libx264 -crf {quality[current_crf]} {out}"
    subprocess.call(to_avc, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame= 1500 fps= 10 q=-1.0 Lsize=  230840kB time=00:00:29.94 bitrate=63160.8kbits/s speed= 0.2x          
video:230817kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.009670%
[libx264 @ 0x55ac1e026180] frame I:6     Avg QP:23.78  size:595205
[libx264 @ 0x55ac1e026180] frame P:378   Avg QP:26.23  size:293835
[libx264 @ 0x55ac1e026180] frame B:1116  Avg QP:28.14  size:109064
[libx264 @ 0x55ac1e026180] consecutive B-frames:  0.8%  0.0%  0.0% 99.2%
[libx264 @ 0x55ac1e026180] mb I  I16..4:  7.4% 73.2% 19.4%
[libx264 @ 0x55ac1e026180] mb P  I16..4:  0.1% 22.1%  8.6%  P16..4: 19.5% 12.9%  5.6%  0.0%  0.0%    skip:31.2%
[libx264 @ 0x55ac1e026180] mb B  I16..4:  0.0%  1.3%  0.5%  B16..8: 27.8% 10.6%  3.6%  direct: 8.1%  skip:48.1%  L0:33.0% L1:35.1% BI:31.9%
[libx264 @ 0x55ac1e026180] 8x8 transform intra:71.5% inter:71.3%
[libx264 @ 0x55ac1e026180] coded y,uvDC,uvAC intra: 95.1% 19.3% 1.2% inter: 26.0% 1.9% 0.0%
[libx264 @ 0x55ac1e026180] i16 v,h,dc,p:  2% 50%  

frame= 1500 fps= 18 q=-1.0 Lsize=   43442kB time=00:00:29.94 bitrate=11886.3kbits/s speed=0.354x    
video:43423kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.043269%
[libx264 @ 0x560dfc463180] frame I:6     Avg QP:35.71  size:156108
[libx264 @ 0x560dfc463180] frame P:378   Avg QP:38.75  size: 59801
[libx264 @ 0x560dfc463180] frame B:1116  Avg QP:41.67  size: 18749
[libx264 @ 0x560dfc463180] consecutive B-frames:  0.8%  0.0%  0.0% 99.2%
[libx264 @ 0x560dfc463180] mb I  I16..4: 22.9% 66.4% 10.7%
[libx264 @ 0x560dfc463180] mb P  I16..4:  2.7% 11.6%  0.5%  P16..4: 28.3%  5.1%  2.3%  0.0%  0.0%    skip:49.4%
[libx264 @ 0x560dfc463180] mb B  I16..4:  0.1%  0.3%  0.0%  B16..8: 25.9%  1.8%  0.1%  direct: 0.3%  skip:71.5%  L0:44.3% L1:53.7% BI: 2.0%
[libx264 @ 0x560dfc463180] 8x8 transform intra:77.2% inter:93.0%
[libx264 @ 0x560dfc463180] coded y,uvDC,uvAC intra: 47.5% 3.4% 0.1% inter: 6.8% 0.0% 0.0%
[libx264 @ 0x560dfc463180] i16 v,h,dc,p:  2% 87%  3%  8%
[

## From YUV multires to H265 HEVC

In [3]:
import os
import subprocess
source = "forest"
path = "test_vids/refs/" + source + ".yuv"
res_width = 3840
res_height = 2160
quality = [15, 22, 28, 34, 40]
for current_crf in range(0,5):
    out = "test_vids/encoded/" + source + "/" + source + "HEVC_cf" + str(quality[current_crf]) + ".mp4"
    to_avc = f"ffmpeg -f rawvideo -video_size {res_width}x{res_height} \
    -r 50 -pixel_format yuv420p -i {path} -c:v libx265 -crf {quality[current_crf]} {out}"
    subprocess.call(to_avc, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame= 1500 fps=8.8 q=35.5 Lsize=   52087kB time=00:00:29.94 bitrate=14251.6kbits/s speed=0.176x    
video:52064kB audio:0kB subtitle:0kB other streams:0kB global headers:2kB muxing overhead: 0.043473%
x265 [info]: frame I:      6, Avg QP:26.65  kb/s: 260536.60
x265 [info]: frame P:    383, Avg QP:28.71  kb/s: 46654.96
x265 [info]: frame B:   1111, Avg QP:37.21  kb/s: 1702.01 
x265 [info]: Weighted P-Frames: Y:0.0% UV:0.0%
x265 [info]: consecutive B-frames: 2.8% 0.8% 5.1% 90.5% 0.8% 

encoded 1500 frames in 170.32s (8.81 fps), 14215.34 kb/s, Avg QP:35.00
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --

## From YUV multires to VP9 WEBM

In [4]:
import os
import subprocess
source = "smithy"
path = "test_vids/refs/" + source + ".yuv"
res_width = 3840
res_height = 2160
quality = [15, 20, 25, 30, 35]
for current_crf in range(0,5):
    out = "test_vids/encoded/" + source + "/" + source + "VP9_cf" + str(quality[current_crf]) + ".webm"
    to_avc = f"ffmpeg -f rawvideo -video_size {res_width}x{res_height} \
    -r 50 -pixel_format yuv420p -i {path} -c:v libvpx-vp9 -crf {quality[current_crf]} {out}"
    subprocess.call(to_avc, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame= 1500 fps=1.9 q=35.0 Lsize=  108292kB time=00:00:29.98 bitrate=29589.6kbits/s speed=0.0374x    
video:108279kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.011628%
